In [3]:
import skweak
import spacy
import re
import os
import numpy as np
import pandas as pd

In [4]:
def resgate():
    pdf = open('contratos.txt', 'r')    
    leitura = pdf.read().replace("\n", "")
    regex_ = re.compile(r"(EXTRATO\sD[O|E]\sCONTRATO[.|\s|\S]*?<>END OF BLOCK<>)")
    return(regex_.findall(leitura))

dados = resgate()

In [5]:
# nlp = spacy.load('en_core_web_sm', disable=["ner", "lemmatizer"])
nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
docs = list(nlp.pipe(dados))

In [6]:
def contrato_(doc):
    expression = r"(\d+/\d{4})"
    for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            tamanho = len(grupo.split())
            for token in docs:
                if token.text in {grupo}:
                    yield token.i, token.i+tamanho, "CONTRATO"
                    # print(docs[0][token.i:token.i+tamanho])
            break 

def processo_(doc):
    expression = r"[P|p][R|r][O|o][C|c][E|e][S|s][S|s][O|o][:|\sSEI].*?(\d+[\.|-|-|\/]\d+[\.|-|-|\/]\d+[\.|-|-|\/]\d*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'\/\s', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "PROCESSO"
                    break
            break

def data_assinatura_(doc):
    expression = r"[A|a][S|s][S|s][I|i][N|n][A|a][T|t][U|u][R|r][A|a]:.*?[\s\S](\d{2}\/\d{2}\/\d{4}|\d+.*?[\s\S]\w+.*?[\s\S]\w+.*?[\s\S]\w+.*?[\s\S]\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'\/\s', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "DATA ASSINAT."
                    break
            break

# def partes_(doc):
#     expression = r"[P|p][A|a][R|r][T|t][E|e][S|s]:.?([^,|;|]*)"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             if ("(" or ")") in grupo:
#                     grupo = grupo.replace("(", "\(")
#                     grupo = grupo.replace(")", "\)")
#             start = re.search(grupo, docs[0].text).span()[0]
#             end = re.search(grupo, docs[0].text).span()[1]
#             span = docs[0].char_span(start, end)
#             print("Found match:", span.text)
#             break

# def objeto_(doc):
#     expression = r"[O|o][B|b][J|j][E|e][T|t][O|o]:.?([^,|;|.]*)"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             if ("(" or ")") in grupo:
#                     grupo = grupo.replace("(", "\(")
#                     grupo = grupo.replace(")", "\)")
#             start = re.search(grupo, docs[0].text).span()[0]
#             end = re.search(grupo, docs[0].text).span()[1]
#             span = docs[0].char_span(start, end)
#             print("Found match:", span.text)
#             break


def vigencia_(doc):
    expression = r"[V|v][I|i][G|g][E|e|ê][N|n][C|c][I|i][A|a].*?[\S\s].*?([^,|;|.]*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "VIGÊNCIA"
                    break
            break

def valor_(doc):
    expression = r"[v|V][a|A][l|L][o|O][r|R].*?[\s\S].*?([\d\.]*,\d{2})"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "VALOR"
                    break
            break

def unidade_orcamento_(doc):
    expression = r"[o|O][r|R][c|C|ç|Ç][a|A][m|M][e|E][n|N][t|T][a|A|á|Á][r|R][i|I][a|A].*?[\s\S].*?(\d+.\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "UN. ORCAMENTARIA"
                    break
            break

def programa_trabalho_(doc):
    expression = r"[T|t][R|r][A|a][B|b][A|a][L|l][H|h][O|o]?[:|;|[\s\S].*?(\d*.\d*.\d*.\d*.\d{4,6}.\d{4,6})"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "PROG. TRABALHO"
                    break
            break

def natureza_emepnho_(doc):
    expression = r"Natureza d[e|a] Despesa:[\s\S].*?([\d.\d]*)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "NAT. EMPENHO"
                    break
            break

def nota_empenho_(doc):
    expression = r"(\d+NE\d+)"
    for match in  re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
        if(match.groups):
            grupo = (match.groups()[0])
            grupo_copy = grupo
            if ("(" or ")") in grupo:
                    grupo = grupo.replace("(", "\(")
                    grupo = grupo.replace(")", "\)")
            tamanho = len(grupo.split())
            start = re.search(grupo, doc.text).span()[0]
            end = re.search(grupo, doc.text).span()[1]
            span = str(doc.char_span(start, end))
            x = re.findall(r'[\(|\)]', span)
            for token in doc:
                if(grupo_copy in str(doc[token.i: token.i+tamanho+len(x)])):
                    yield token.i, token.i+tamanho+len(x), "NOTA EMPENHO"
                    break
            break


# def entidade_contratada_(doc):
#     expression = r"[c|C][o|O][n|N][t|T][r|R][a|A][t|T][a|A][d|D][a|A|o|O].(.+\w+[\s\S].*?[\/.|])"
#     for match in re.finditer(expression, doc.text): #doc.tex with m re.finditer to return span with char_span
#         if(match.groups):
#             grupo = (match.groups()[0])
#             if ("(" or ")") in grupo:
#                     grupo = grupo.replace("(", "\(")
#                     grupo = grupo.replace(")", "\)")
#             start = re.search(grupo, docs[0].text).span()[0]
#             end = re.search(grupo, docs[0].text).span()[1]
#             span = docs[0].char_span(start, end)
#             print("Found match:", span.text)
#             break


In [7]:
detec_contrato = skweak.heuristics.FunctionAnnotator("detec_contrato", contrato_)
detec_processo = skweak.heuristics.FunctionAnnotator("detec_processo", processo_)
detec_data = skweak.heuristics.FunctionAnnotator("detec_data", data_assinatura_)
detec_vigencia = skweak.heuristics.FunctionAnnotator("detec_vigencia", vigencia_)
detec_valor = skweak.heuristics.FunctionAnnotator("detec_valor", valor_)
detec_unidade = skweak.heuristics.FunctionAnnotator("detec_unidade", unidade_orcamento_)
detec_programa = skweak.heuristics.FunctionAnnotator("detec_programa", programa_trabalho_)
detec_natureza = skweak.heuristics.FunctionAnnotator("detec_natureza", natureza_emepnho_)
detec_nota = skweak.heuristics.FunctionAnnotator("detec_nota", nota_empenho_)

In [8]:
docs = list(detec_contrato.pipe(docs))
docs = list(detec_processo.pipe(docs))
docs = list(detec_data.pipe(docs))
docs = list(detec_vigencia.pipe(docs))
docs = list(detec_valor.pipe(docs))
docs = list(detec_unidade.pipe(docs))
docs = list(detec_programa.pipe(docs))
docs = list(detec_natureza.pipe(docs))
docs = list(detec_nota.pipe(docs))

In [9]:
# docsAux = list(detec_processo.pipe(docs))
skweak.utils.display_entities(docs[0], ["detec_contrato", "detec_processo", "detec_valor", "detec_data", "detec_vigencia", "detec_unidade", "detec_programa","detec_natureza","detec_nota"])

In [10]:
model = skweak.aggregation.HMM("hmm", ["CONTRATO", "PROCESSO", "DATA ASSINAT.", "VIGENCIA", "VALOR", "UN. ORCAMENTARIA", "PROG. TRABALHO", "NAT. EMPENHO", "NOTA EMPENHO"])
docs = model.fit_and_aggregate(docs)

Starting iteration 1
Finished E-step with 54 documents
Starting iteration 2


         1       -1973.8169             +nan


Finished E-step with 54 documents
Starting iteration 3


         2       -1966.1279          +7.6891


Finished E-step with 54 documents
Starting iteration 4


         3       -1964.2490          +1.8789


Finished E-step with 54 documents
Starting iteration 5


         4       -1960.2704          +3.9785


Finished E-step with 54 documents


         5       -1955.1972          +5.0732


In [12]:
for doc in docs:
    doc.ents = doc.spans["hmm"]
skweak.utils.docbin_writer(docs, "./data/reuters_small.spacy")

Write to ./data/reuters_small.spacy...done


In [13]:
from spacy.tokens import DocBin
nlp = spacy.blank("pt")
doc_bin = DocBin().from_disk("./data/reuters_small.spacy")  # your file here
examples = []  # examples in Prodigy's format
for doc in doc_bin.get_docs(nlp.vocab):
    lael_iob = ""
    for i in range(0, len(doc)):
        _label_ = doc[i].ent_iob_
        _ent_ = doc[i].ent_type_
        if(_label_ != "O"):
            lael_iob+=f'{_label_}-{_ent_} '
        else:
            lael_iob+=f'{_label_} '
    spans = [{"start": ent.start_char, "end": ent.end_char, "label": ent.label_} for ent in doc.ents]
    examples.append({"text": doc.text, "iob": lael_iob,"spans": spans})

In [14]:
examples[0]["iob"]

'O O O O O O O O O O O O O B-PROCESSO O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-VALOR O O O O O O O O O O O O O O O O O O O O O O O O O O B-UN. ORCAMENTARIA O O O O O O O O O O O O O O O O O O O B-NOTA EMPENHO O O O O O O O O O O O O O O O O O O O O O O B-DATA ASSINAT. I-DATA ASSINAT. I-DATA ASSINAT. I-DATA ASSINAT. I-DATA ASSINAT. O O O O O O O O O O O O O O O O O O O O O O O O O O O O '

In [15]:
examples[0]

{'text': 'EXTRATO DO CONTRATO No 29/2013, NOS TERMOS DO PADRAO No 09/2012.Processo: 139.000.604/2013. Partes: DF/RAXI x SUMMIT CONSTRUCOES E TECNOLOGIA LTDA.-EPP, Fundamento Legal: Convite no 028/2013-RA XI. Objeto: Contratacao de Empresa para Execucao de Reforma e Instalacoes Eletrica da Feira Permanente do Cruzeiro, conforme especificacoes tecnicas do convite e proposta que passam a integrar o presente Termo. Valor: O Valor total do Contrato e de R$ 145.832,25 (cento e quarenta e cinco mil, oitocentos e trinta e dois reais e vinte e cinco centavos), Dotacao orcamentaria: U.O: 09113. Programa de Traba-lho: 15.451.6208.1110.9795 Natureza de Despesa: 449051. Fonte de Recurso: 100. Empenho 2013NE00430. Vigencia: O presente Termo Aditivo entrara em vigor a partir da data da sua assinatura. Data da assinatura: 31 de outubro de 2013. Signatarios: Pelo DF, Antonio Sabino de Vasconcelos Neto, e pela Contratada, Celia Regina Assencio Carvalho. <>END OF BLOCK<>',
 'iob': 'O O O O O O O O O O O 